In [2]:
# Importar librerías
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Librerías importadas correctamente ✓")

Librerías importadas correctamente ✓


## 1. Cargar datos de EPH

In [3]:
def cargar_datos_eph_individuos(directorio_raw: str = "../datos/raw") -> pd.DataFrame:
    """
    Carga archivos de EPH individuos (no hogares)
    """
    directorio = Path(directorio_raw)
    subdirectorios = [d for d in directorio.iterdir() if d.is_dir()]
    
    print(f"Buscando archivos de individuos en {len(subdirectorios)} carpetas...")
    
    dataframes = []
    
    for subdir in sorted(subdirectorios):
        # Buscar todos los archivos .txt
        archivos_txt = list(subdir.glob("*.txt"))
        
        for archivo in archivos_txt:
            try:
                # Leer primero solo las columnas para verificar
                df_temp = pd.read_csv(archivo, sep=';', encoding='latin-1', nrows=1)
                
                # Verificar si es archivo de individuos (tiene CH04 o ESTADO)
                if 'CH04' in df_temp.columns or 'ESTADO' in df_temp.columns:
                    print(f"✓ Cargando individuos: {archivo.name}...", end=" ")
                    df = pd.read_csv(archivo, sep=';', encoding='latin-1', low_memory=False)
                    dataframes.append(df)
                    print(f"({len(df):,} registros)")
                    
            except Exception as e:
                continue
    
    if dataframes:
        df_consolidado = pd.concat(dataframes, ignore_index=True)
        print(f"\n{'='*60}")
        print(f"Total individuos: {len(df_consolidado):,}")
        print(f"{'='*60}")
        return df_consolidado
    else:
        raise ValueError("No se encontraron archivos de individuos")

# Cargar datos de individuos
df_eph = cargar_datos_eph_individuos()

Buscando archivos de individuos en 40 carpetas...
✓ Cargando individuos: usu_individual_t217.txt... (59,755 registros)
✓ Cargando individuos: usu_individual_t317.txt... (58,721 registros)
✓ Cargando individuos: Usu_Individual_T417.txt... (58,181 registros)
✓ Cargando individuos: usu_individual_t118.txt... (57,951 registros)
✓ Cargando individuos: usu_individual_t218.txt... (57,835 registros)
✓ Cargando individuos: usu_individual_T318.txt... (56,879 registros)
✓ Cargando individuos: usu_individual_t418.txt... (57,418 registros)
✓ Cargando individuos: usu_individual_t119.txt... (59,369 registros)
✓ Cargando individuos: USU_individual_t219.txt... (59,258 registros)
✓ Cargando individuos: Usu_individual_T319.txt... (57,229 registros)
✓ Cargando individuos: usu_individual_T419.txt... (58,519 registros)
✓ Cargando individuos: usu_individual_T120.txt... (51,643 registros)
✓ Cargando individuos: usu_Individual_T220.txt... (37,132 registros)
✓ Cargando individuos: usu_individual_T320.txt... (41

## 2. Exploración inicial de los datos

In [3]:
# Información general
print("Dimensiones del dataset:", df_eph.shape)
print("\nPrimeras columnas:")
print(df_eph.columns.tolist()[:30])

Dimensiones del dataset: (1143475, 239)

Primeras columnas:
['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'CH03', 'CH04', 'CH05', 'CH06', 'CH07', 'CH08', 'CH09', 'CH10', 'CH11', 'CH12', 'CH13', 'CH14', 'CH15', 'CH15_COD', 'CH16', 'CH16_COD', 'NIVEL_ED', 'ESTADO', 'CAT_OCUP', 'CAT_INAC']


In [4]:
# Variables clave para el análisis
variables_clave = [
    'CODUSU',      # Código de vivienda
    'ANO4',        # Año
    'TRIMESTRE',   # Trimestre
    'AGLOMERADO',  # Código de aglomerado
    'REGION',      # Región
    'CH04',        # Sexo (1=Varón, 2=Mujer)
    'CH06',        # Edad
    'NIVEL_ED',    # Nivel educativo
    'ESTADO',      # Estado (1=Ocupado, 2=Desocupado, 3=Inactivo, 4=Menor de 10 años)
    'CAT_OCUP',    # Categoría ocupacional
    'PP04B_COD',   # Código de actividad
    'PP04D_COD',   # Código de ocupación
    'P21',         # Ingreso ocupación principal
    'P47T',        # Ingreso total individual
    'ITF',         # Ingreso total familiar
    'PONDERA'      # Ponderador
]

# Verificar cuáles variables existen
variables_disponibles = [v for v in variables_clave if v in df_eph.columns]
variables_faltantes = [v for v in variables_clave if v not in df_eph.columns]

print("Variables disponibles:", len(variables_disponibles))
print("Variables faltantes:", variables_faltantes if variables_faltantes else "Ninguna")

Variables disponibles: 16
Variables faltantes: Ninguna


In [5]:
# Seleccionar solo las variables disponibles
df_trabajo = df_eph[variables_disponibles].copy()

print(f"Dataset de trabajo: {df_trabajo.shape}")
print(f"\nPrimeros registros:")
df_trabajo.head()

Dataset de trabajo: (1143475, 16)

Primeros registros:


,CODUSU,ANO4,TRIMESTRE,AGLOMERADO,REGION,CH04,CH06,NIVEL_ED,ESTADO,CAT_OCUP,PP04B_COD,PP04D_COD,P21,P47T,ITF,PONDERA
0,TQRMNORWQHMMLNCDEFIAH00507965,2017,2,2,43,1,71,2,3,0.0,NaN,NaN,0,5900.0,21800,504
1,TQRMNORWQHMMLNCDEFIAH00507965,2017,2,2,43,2,66,2,3,0.0,NaN,NaN,0,5900.0,21800,504
2,TQRMNORWQHMMLNCDEFIAH00507965,2017,2,2,43,1,20,4,1,3.0,4502.0,82313.0,10000,10000.0,21800,504
3,TQSMNORWQHMMLNCDEFIAH00507893,2017,2,2,43,2,47,2,3,0.0,NaN,NaN,0,1900.0,15900,504
4,TQSMNORWQHMMLNCDEFIAH00507893,2017,2,2,43,1,43,4,1,3.0,4000.0,72313.0,14000,14000.0,15900,504


## 3. Crear variables derivadas e indicadores

In [6]:
# Filtrar solo personas de 10 años o más (población en edad de trabajar)
df_trabajo = df_eph[df_eph['CH06'] >= 10].copy()

print(f"Registros totales: {len(df_eph):,}")
print(f"Registros de población +10 años: {len(df_trabajo):,}")

# Crear variable de período
df_trabajo['periodo'] = df_trabajo['ANO4'].astype(str) + '-T' + df_trabajo['TRIMESTRE'].astype(str)

# Crear variable de sexo descriptiva
df_trabajo['sexo'] = df_trabajo['CH04'].map({1: 'Varón', 2: 'Mujer'})

# Crear categorías de edad
df_trabajo['grupo_edad'] = pd.cut(
    df_trabajo['CH06'],
    bins=[0, 18, 25, 35, 45, 55, 65, 120],
    labels=['0-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
)

# Crear variable de condición de actividad
df_trabajo['condicion_actividad'] = df_trabajo['ESTADO'].map({
    1: 'Ocupado',
    2: 'Desocupado', 
    3: 'Inactivo',
    4: 'Menor de 10 años'
})

# Crear indicadores de PEA (Población Económicamente Activa)
df_trabajo['es_pea'] = df_trabajo['ESTADO'].isin([1, 2])
df_trabajo['es_ocupado'] = df_trabajo['ESTADO'] == 1
df_trabajo['es_desocupado'] = df_trabajo['ESTADO'] == 2

print("\nVariables derivadas creadas ✓")
print(f"\nDistribución de condición de actividad:")
print(df_trabajo['condicion_actividad'].value_counts())

Registros totales: 1,143,475
Registros de población +10 años: 987,193

Variables derivadas creadas ✓

Distribución de condición de actividad:
condicion_actividad
Ocupado       476423
Inactivo      471108
Desocupado     38178
Name: count, dtype: int64


## 4. Calcular tasas del mercado laboral por período

In [7]:
def calcular_tasas_laborales(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula las tasas de actividad, empleo y desocupación por período
    
    Args:
        df: DataFrame con datos de EPH
        
    Returns:
        DataFrame con las tasas calculadas por período
    """
    # Filtrar población en edad de trabajar (10+ años)
    df_pet = df[df['CH06'] >= 10].copy()
    
    tasas = []
    
    for periodo in sorted(df_pet['periodo'].unique()):
        df_periodo = df_pet[df_pet['periodo'] == periodo]
        
        # Población total en edad de trabajar
        pet = df_periodo['PONDERA'].sum()
        
        # Población económicamente activa (PEA)
        pea = df_periodo[df_periodo['es_pea']]['PONDERA'].sum()
        
        # Ocupados
        ocupados = df_periodo[df_periodo['es_ocupado']]['PONDERA'].sum()
        
        # Desocupados
        desocupados = df_periodo[df_periodo['es_desocupado']]['PONDERA'].sum()
        
        # Calcular tasas
        tasa_actividad = (pea / pet) * 100 if pet > 0 else 0
        tasa_empleo = (ocupados / pet) * 100 if pet > 0 else 0
        tasa_desocupacion = (desocupados / pea) * 100 if pea > 0 else 0
        
        tasas.append({
            'periodo': periodo,
            'tasa_actividad': tasa_actividad,
            'tasa_empleo': tasa_empleo,
            'tasa_desocupacion': tasa_desocupacion,
            'pea': pea,
            'ocupados': ocupados,
            'desocupados': desocupados
        })
    
    return pd.DataFrame(tasas)

# Calcular tasas
df_tasas = calcular_tasas_laborales(df_trabajo)

print("Tasas laborales calculadas ✓")
print(f"\nPrimeros registros:")
df_tasas.head(10)

Tasas laborales calculadas ✓

Primeros registros:


,periodo,tasa_actividad,tasa_empleo,tasa_desocupacion,pea,ocupados,desocupados
0,2017-T2,53.340230,48.689356,8.719262,12483270,11394821,1088449
1,2017-T3,54.088860,49.600311,8.298472,12750829,11692705,1058124
2,2017-T4,54.266677,50.348234,7.220717,12817952,11892404,925548
3,2018-T1,54.547965,49.558153,9.147568,12931842,11748893,1182949
4,2018-T2,54.247990,49.030104,9.618579,12881342,11642340,1239002
5,2018-T3,54.480441,49.582675,8.989953,12989801,11822024,1167777
6,2018-T4,54.318192,49.358243,9.131286,12978501,11793397,1185104
7,2019-T1,54.611456,49.110972,10.072034,13285430,11947317,1338113
8,2019-T2,55.350381,49.459904,10.642162,13511286,12073393,1437893
9,2019-T3,55.051519,49.690959,9.737351,13220865,11933503,1287362


## 5. Ajuste de ingresos por inflación

Los ingresos nominales deben ser ajustados por inflación para poder comparar el poder adquisitivo a lo largo del tiempo.

In [8]:
# Índice de Precios al Consumidor (IPC) - Base 2016 = 100
# NOTA: Estos son valores aproximados. Deberían actualizarse con datos reales del INDEC
# Para el análisis final, usar los datos oficiales de IPC del INDEC

ipc_data = {
    '2016-T1': 100.0,
    '2016-T2': 110.5,
    '2016-T3': 118.3,
    '2016-T4': 125.2,
    '2017-T1': 133.8,
    '2017-T2': 138.9,
    '2017-T3': 144.5,
    '2017-T4': 151.7,
    '2018-T1': 160.2,
    '2018-T2': 172.5,
    '2018-T3': 198.6,
    '2018-T4': 221.3,
    '2019-T1': 243.7,
    '2019-T2': 268.4,
    '2019-T3': 297.1,
    '2019-T4': 328.9,
    '2020-T1': 352.4,
    '2020-T2': 366.8,
    '2020-T3': 385.2,
    '2020-T4': 408.7,
    '2021-T1': 433.5,
    '2021-T2': 459.8,
    '2021-T3': 490.3,
    '2021-T4': 528.6,
    '2022-T1': 576.4,
    '2022-T2': 630.2,
    '2022-T3': 705.8,
    '2022-T4': 801.5,
    '2023-T1': 915.3,
    '2023-T2': 1089.7,
    '2023-T3': 1287.4,
    '2023-T4': 1556.8,
    '2024-T1': 1923.5,
    '2024-T2': 2187.3,
    '2024-T3': 2398.6,
    '2024-T4': 2587.2,
    '2025-T1': 2756.8,
    '2025-T2': 2891.4,
    '2025-T3': 3012.7,
    '2025-T4': 3125.3,
}

# Crear DataFrame de IPC
df_ipc = pd.DataFrame(list(ipc_data.items()), columns=['periodo', 'ipc'])

# Período base para ajuste (último trimestre disponible)
periodo_base = df_ipc['periodo'].max()
ipc_base = df_ipc[df_ipc['periodo'] == periodo_base]['ipc'].values[0]

print(f"Período base para ajuste: {periodo_base}")
print(f"IPC base: {ipc_base}")
print(f"\nÍndice IPC por período:")
print(df_ipc.tail(10))

Período base para ajuste: 2025-T4
IPC base: 3125.3

Índice IPC por período:
    periodo     ipc
30  2023-T3  1287.4
31  2023-T4  1556.8
32  2024-T1  1923.5
33  2024-T2  2187.3
34  2024-T3  2398.6
35  2024-T4  2587.2
36  2025-T1  2756.8
37  2025-T2  2891.4
38  2025-T3  3012.7
39  2025-T4  3125.3


In [9]:
# Agregar IPC al dataset de trabajo
df_trabajo = df_trabajo.merge(df_ipc, on='periodo', how='left')

# Calcular factor de ajuste
df_trabajo['factor_ajuste'] = ipc_base / df_trabajo['ipc']

# Ajustar ingresos (si las columnas existen)
columnas_ingreso = ['P21', 'P47T', 'ITF']

for col in columnas_ingreso:
    if col in df_trabajo.columns:
        # Convertir a numérico primero (los valores no numéricos se convierten en NaN)
        df_trabajo[col] = pd.to_numeric(df_trabajo[col], errors='coerce')
        
        # Calcular ingreso real ajustado
        df_trabajo[f'{col}_real'] = df_trabajo[col] * df_trabajo['factor_ajuste']
        print(f"✓ Ingreso ajustado creado: {col}_real")

print("\nIngresos ajustados por inflación ✓")

# Mostrar ejemplo de ajuste
print("\nEjemplo de ajuste (primeros 5 registros con ingreso):")
ejemplo = df_trabajo[df_trabajo['P21'].notna()][['periodo', 'P21', 'ipc', 'P21_real']].head()
print(ejemplo)

✓ Ingreso ajustado creado: P21_real
✓ Ingreso ajustado creado: P47T_real
✓ Ingreso ajustado creado: ITF_real

Ingresos ajustados por inflación ✓

Ejemplo de ajuste (primeros 5 registros con ingreso):
   periodo      P21    ipc       P21_real
0  2017-T2      0.0  138.9       0.000000
1  2017-T2      0.0  138.9       0.000000
2  2017-T2  10000.0  138.9  225003.599712
3  2017-T2      0.0  138.9       0.000000
4  2017-T2  14000.0  138.9  315005.039597


## 6. Guardar datos procesados

In [11]:
# Crear directorio de salida si no existe
Path("../datos/processed").mkdir(parents=True, exist_ok=True)

print("Normalizando tipos de datos para exportar a Parquet...")

# Forzar conversión de todas las columnas object a string primero
# Esto elimina el problema de tipos mixtos (int y string en la misma columna)
for col in df_trabajo.columns:
    if df_trabajo[col].dtype == 'object':
        # Convertir todo a string
        df_trabajo[col] = df_trabajo[col].astype(str)
        # Intentar convertir de vuelta a numérico si todos los valores son numéricos
        df_trabajo[col] = pd.to_numeric(df_trabajo[col], errors='ignore')

print("✓ Tipos normalizados")

# Guardar dataset principal procesado
ruta_salida_principal = "../datos/processed/eph_consolidado.parquet"
df_trabajo.to_parquet(ruta_salida_principal, index=False, compression='snappy')
print(f"✓ Dataset principal guardado: {ruta_salida_principal}")
print(f"  Dimensiones: {df_trabajo.shape}")

# Guardar también en CSV por si acaso
ruta_salida_csv = "../datos/processed/eph_consolidado.csv"
df_trabajo.to_csv(ruta_salida_csv, index=False, encoding='utf-8-sig')
print(f"✓ Dataset también guardado en CSV: {ruta_salida_csv}")

print("\n✓ Datos procesados y guardados correctamente")

Normalizando tipos de datos para exportar a Parquet...
✓ Tipos normalizados
✓ Dataset principal guardado: ../datos/processed/eph_consolidado.parquet
  Dimensiones: (987193, 251)
✓ Dataset también guardado en CSV: ../datos/processed/eph_consolidado.csv

✓ Datos procesados y guardados correctamente


## 7. Resumen de datos procesados

In [10]:
print("="*60)
print("RESUMEN DE DATOS PROCESADOS")
print("="*60)

print(f"\nTotal de registros: {len(df_trabajo):,}")
print(f"Períodos analizados: {df_trabajo['periodo'].nunique()}")
print(f"Rango temporal: {df_trabajo['periodo'].min()} a {df_trabajo['periodo'].max()}")

print(f"\nAglomerados únicos: {df_trabajo['AGLOMERADO'].nunique() if 'AGLOMERADO' in df_trabajo.columns else 'N/A'}")

print(f"\nVariables en el dataset final: {len(df_trabajo.columns)}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df_trabajo.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n{'='*60}")
print("Datos listos para análisis ✓")
print(f"{'='*60}")

RESUMEN DE DATOS PROCESADOS

Total de registros: 987,193
Períodos analizados: 22
Rango temporal: 2017-T2 a 2025-T2

Aglomerados únicos: 32

Variables en el dataset final: 251

Columnas disponibles:
   1. CODUSU
   2. ANO4
   3. TRIMESTRE
   4. NRO_HOGAR
   5. COMPONENTE
   6. H15
   7. REGION
   8. MAS_500
   9. AGLOMERADO
  10. PONDERA
  11. CH03
  12. CH04
  13. CH05
  14. CH06
  15. CH07
  16. CH08
  17. CH09
  18. CH10
  19. CH11
  20. CH12
  21. CH13
  22. CH14
  23. CH15
  24. CH15_COD
  25. CH16
  26. CH16_COD
  27. NIVEL_ED
  28. ESTADO
  29. CAT_OCUP
  30. CAT_INAC
  31. IMPUTA
  32. PP02C1
  33. PP02C2
  34. PP02C3
  35. PP02C4
  36. PP02C5
  37. PP02C6
  38. PP02C7
  39. PP02C8
  40. PP02E
  41. PP02H
  42. PP02I
  43. PP03C
  44. PP03D
  45. PP3E_TOT
  46. PP3F_TOT
  47. PP03G
  48. PP03H
  49. PP03I
  50. PP03J
  51. INTENSI
  52. PP04A
  53. PP04B_COD
  54. PP04B1
  55. PP04B2
  56. PP04B3_MES
  57. PP04B3_ANO
  58. PP04B3_DIA
  59. PP04C
  60. PP04C99
  61. PP04D_COD
  6

In [ ]:
# Guardar archivos adicionales para notebooks de análisis

# 1. Guardar tasas laborales
df_tasas.to_csv("../datos/processed/tasas_laborales.csv", index=False, encoding='utf-8-sig')
print(f"✓ Tasas laborales guardadas: {len(df_tasas)} períodos")

# 2. Guardar IPC
df_ipc.to_csv("../datos/processed/ipc.csv", index=False, encoding='utf-8-sig')
print(f"✓ IPC guardado: {len(df_ipc)} períodos")

print("\n✓ Archivos adicionales creados para análisis posterior")